# Timer

In [5]:
from time import sleep
from datetime import datetime

month, day, hour, minute = 10, 22, 13, 30

while True:
    dt = datetime.now()
    if dt.month >= month and dt.day >= day:
        if dt.hour > hour or (dt.hour == hour and dt.minute > minute):
            break
    sleep(60)

# Parameter optimization

In [6]:
import sys
import pandas as pd
import numpy as np
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from bot.bot import SigBot
from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'sell'
pattern = ['STOCH', 'RSI']
work_timeframe = '15m'
higher_timeframe = '4h'
opt_limit = 300
load = False

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {'RSI': {'timeperiod': [12, 14, 16], 'low_bound': [20, 25, 30, 35]},
              'STOCH': {'fastk_period': [3, 5, 7, 9], 'slowk_period': [2, 3, 4, 5],
                        'slowd_period': [2, 3, 5, 7], 'low_bound': [10, 15, 20, 25]}}
# optim_dict = {'RSI': {'timeperiod': [16], 'low_bound': [30]},
#                   'STOCH': {'fastk_period': [5], 'slowk_period': [4],
#                             'slowd_period': [2], 'low_bound': [30]}}

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['indicator_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat = stat.sort_values(['indicator_rank', 'pct_right_forecast_avg'], ascending=False)
stat.head(20)

Number of combinations is 3072


  0%|          | 0/3072 [00:00<?, ?it/s]

,pattern,RSI_timeperiod,RSI_low_bound,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,pct_right_forecast_1,pct_right_forecast_2,pct_right_forecast_3,pct_right_forecast_4,pct_right_forecast_5,pct_right_forecast_6,pct_right_forecast_7,pct_right_forecast_8,pct_right_forecast_9,pct_right_forecast_10,pct_right_forecast_11,pct_right_forecast_12,pct_right_forecast_13,pct_right_forecast_14,pct_right_forecast_15,pct_right_forecast_16,pct_right_forecast_17,pct_right_forecast_18,pct_right_forecast_19,pct_right_forecast_20,pct_right_forecast_21,pct_right_forecast_22,pct_right_forecast_23,pct_right_forecast_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,pct_right_forecast_avg,indicator_rank
1809,STOCH_RSI,14,35,3,3,2,15,92.50,87.50,90.00,77.50,77.50,77.50,80.00,80.00,80.00,82.50,77.50,82.50,72.50,70.00,65.00,67.50,72.50,72.50,65.00,70.00,67.50,65.00,65.00,62.50,-0.20,-0.37,-0.46,-0.35,-0.37,-0.37,-0.49,-0.57,-0.67,-0.69,-0.65,-0.57,-0.68,-0.85,-0.72,-0.74,-0.64,-0.51,-0.51,-0.59,-0.55,-0.46,-0.59,-0.71,40,75.000000,200.000000
1808,STOCH_RSI,14,35,3,3,2,10,80.00,100.00,100.00,70.00,70.00,80.00,80.00,90.00,90.00,100.00,100.00,100.00,90.00,90.00,90.00,80.00,90.00,100.00,80.00,90.00,80.00,80.00,80.00,80.00,-0.20,-0.39,-0.55,-0.45,-0.32,-0.28,-0.40,-0.56,-0.69,-0.69,-0.66,-0.67,-0.82,-0.93,-0.94,-0.94,-0.86,-0.77,-0.71,-0.89,-0.91,-0.75,-0.90,-1.15,10,87.083333,170.833333
1798,STOCH_RSI,14,35,3,2,3,20,89.80,83.67,85.71,71.43,73.47,77.55,79.59,79.59,73.47,77.55,73.47,79.59,71.43,71.43,67.35,69.39,73.47,71.43,65.31,69.39,65.31,63.27,63.27,61.22,-0.16,-0.28,-0.43,-0.34,-0.34,-0.30,-0.44,-0.51,-0.65,-0.61,-0.65,-0.58,-0.73,-0.81,-0.66,-0.70,-0.68,-0.54,-0.60,-0.62,-0.56,-0.50,-0.65,-0.82,49,73.215000,157.535000
1872,STOCH_RSI,14,35,5,3,2,10,83.33,91.67,87.50,79.17,75.00,75.00,83.33,79.17,79.17,87.50,79.17,79.17,70.83,75.00,75.00,70.83,70.83,70.83,75.00,70.83,66.67,66.67,70.83,70.83,-0.21,-0.40,-0.66,-0.55,-0.50,-0.52,-0.60,-0.69,-0.71,-0.72,-0.66,-0.74,-0.86,-0.88,-0.84,-0.86,-0.79,-0.69,-0.76,-0.80,-0.85,-0.70,-0.80,-0.84,24,76.388750,153.330000
1810,STOCH_RSI,14,35,3,3,2,20,93.59,85.90,87.18,73.08,73.08,73.08,74.36,75.64,74.36,75.64,71.79,74.36,67.95,70.51,66.67,66.67,69.23,67.95,64.10,66.67,65.38,62.82,62.82,62.82,-0.23,-0.30,-0.39,-0.34,-0.35,-0.34,-0.42,-0.52,-0.59,-0.63,-0.63,-0.58,-0.68,-0.76,-0.64,-0.60,-0.66,-0.55,-0.52,-0.64,-0.66,-0.70,-0.69,-0.68,78,71.902083,148.362500
1796,STOCH_RSI,14,35,3,2,3,10,75.00,100.00,100.00,62.50,62.50,75.00,75.00,87.50,87.50,100.00,100.00,100.00,87.50,87.50,87.50,87.50,87.50,100.00,75.00,87.50,87.50,87.50,87.50,87.50,-0.20,-0.39,-0.55,-0.45,-0.32,-0.19,-0.40,-0.56,-0.69,-0.69,-0.66,-0.67,-0.82,-0.93,-0.94,-0.94,-0.86,-0.77,-0.71,-0.89,-0.91,-0.75,-0.90,-1.15,8,86.458333,131.666667
528,STOCH_RSI,12,30,3,3,2,10,75.00,100.00,100.00,75.00,75.00,75.00,75.00,87.50,87.50,100.00,100.00,100.00,87.50,87.50,87.50,75.00,87.50,100.00,75.00,87.50,75.00,75.00,75.00,75.00,-0.20,-0.39,-0.55,-0.45,-0.32,-0.28,-0.40,-0.56,-0.66,-0.66,-0.65,-0.57,-0.77,-0.85,-0.92,-0.91,-0.85,-0.60,-0.56,-0.77,-0.76,-0.46,-0.67,-0.92,8,84.895833,119.166667
832,STOCH_RSI,12,35,5,2,2,10,78.00,82.00,82.00,76.00,74.00,74.00,76.00,76.00,76.00,84.00,84.00,82.00,78.00,78.00,76.00,68.00,62.00,66.00,66.00,64.00,58.00,56.00,60.00,60.00,-0.12,-0.24,-0.25,-0.27,-0.28,-0.33,-0.41,-0.38,-0.41,-0.46,-0.56,-0.46,-0.58,-0.65,-0.73,-0.52,-0.52,-0.46,-0.44,-0.50,-0.45,-0.44,-0.39,-0.46,50,72.333333,116.666667
1856,STOCH_RSI,14,35,5,2,2,10,74.19,87.10,93.55,80.65,74.19,70.97,74.19,77.42,77.42,87.10,80.65,80.65,74.19,74.19,70.97,70.97,64.52,70.97,64.52,6

# Save new config data to config file

In [3]:
from config_updater import ConfigUpdater

timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {'RSI': {'timeperiod': [14], 'low_bound': [35]},
                  'STOCH': {'fastk_period': [7], 'slowk_period': [2],
                            'slowd_period': [2], 'low_bound': [25]}}
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)